In [1]:
# Load packages

import xarray as xr
import matplotlib
import numpy as np
import os
import re
import pickle

from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import cm
import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import plotting_utils as plu
from sklearn import metrics

%matplotlib inline

In [2]:
# Set paths 

user = 'mollyw'
user_path = '/glade/scratch/' + user + '/'
time_path = '/ice/proc/tseries/day_1/'
stor_path = '/glade/campaign/univ/uwas0070/mwieringa/assimilation/cases/'

# Case List- assimilation

cases = ['DA30_dflt_MAT',
         'DA30_dflt_MAT_err10',
         'DA30_dflt_MAT_mem_0005_err10',
         'DA30_dflt_MAT_mem_0011_err10',
         'DA30_dflt_MAT_mem_0024_noerr',
         'DA30_dflt_MAT_mem_0025_err10',
         'DA30_dflt_MAT_mem_0026_err10',
         'DA30_dflt_MAT_mem_0028_err10',
         'DA30_010_higrid_4_15NOerr',
         'DA30_vicen_MAT_mem_0024_noerr_v6',
         'DA30_vicen_MAT_mem_0005_noerr',
         'DA30_vicen_MAT_mem_0011_noerr',
         'DA30_vicen_MAT_mem_0025_noerr',
         'DA30_vicen_MAT_mem_0026_noerr',
         'DA30_vicen_MAT_mem_0028_noerr',
         
         'DA30_aicen_ITD',
         'DA30_aicen_ITD_loc01',
         'DA30_aicen_ITD_loc10',
         'DA30_aicen_ITD_err05',
         'DA30_aicen_ITD_err01',
         'DA30_aicen_ITD_err10',
         'DA30_aicen_ITD_err15',         
#          'DA30_aicen_ITD_err01_nopert',
#          'DA30_aicen_ITD_err05_nopert',
#          'DA30_aicen_ITD_err15_nopert',
         'DA30_aicen_ITD_loc01_err15',
         'DA30_aicen_ITD_loc05_err15',
         'DA30_aicen_ITD_mem_0005_err15',
         'DA30_aicen_ITD_mem_0011_err15',
         'DA30_aicen_ITD_mem_0024_noerr',
         'DA30_aicen_ITD_mem_0025_err15',
         'DA30_aicen_ITD_mem_0026_err15',
         'DA30_aicen_ITD_mem_0028_err15',
         
         'DA30_dflt_FB_mem_0024_err01_unperturbed',
         'DA30_dflt_FB_loc01_err001',
         'DA30_dflt_FB_loc05_err001',
         'DA30_dflt_FB_err001',
         'DA30_dflt_FB_err005',
         'DA30_dflt_FB_err01',
         'DA30_vicen_FB_mem_0024_noerr',
         'DA30_vicen_FB_loc05_mem_0024_noerr',
         'free30'
        ]

labels = ['MAT',
          'MAT2',
          'MAT_05',
          'MAT_11',
          'MAT_unperturbed',
          'MAT_25',
          'MAT_26',
          'MAT_28',
          'MAT_noerr',
          'MAT_vol_noerr',
          'MAT_vol_05',
          'MAT_vol_11',
          'MAT_vol_25',
          'MAT_vol_26',
          'MAT_vol_28',
          
          'ITD05',
          'ITD01',
          'ITD10',

          'ITD10_err05',
          'ITD10_err01',
          'ITD10_err10',
          'ITD10_err15',
#           'ITD10_err01_nopert',
#           'ITD10_err05_nopert',
#           'ITD10_err15_nopert',
          'ITD01_err15',
          'ITD05_err15',
          'ITD_05',
          'ITD_11',
          'ITD_unperturbed',
          'ITD_25',
          'ITD_26',
          'ITD_28',
          
          'FB_unperturbed',
          'FB_loc01',
          'FB_loc05',
          'FB_err001',
          'FB_err005',
          'FB_err01',
          'FB_vol_noerr',
          'FB_vol_loc05_noerr',
          'FREE'
         ]

In [3]:
experiment={}
variables = ['aice',
             'hi',
             'aicen',
             'vicen',
             'vsnon',
            ]

truths = ['05',
          '11',
          '24',
          '25',
          '26',
          '28',
         ]

for t in truths:
#     print(t)
    if t == '24':
        truth_files = sorted(glob.glob('/glade/campaign/univ/uwas0070/mwieringa/assimilation/archives/free30/ice/hist_perfect/free30.cice_0024.h.2019-04-*[0-9]*'))
        TRUTH_24 = xr.open_mfdataset(truth_files, concat_dim = 'time').sel(time = slice('2019-04-15', '2019-04-30'))
        
        experiment['TRUTH_24'] = TRUTH_24
    else:
        truth_short = sorted(glob.glob(stor_path + '/free30_short/run/free30_short.cice_00{00}'.format(t) + '.h.2019-04-*[0-9]*'))
        truth_long = sorted(glob.glob(stor_path + '/free30_temp/run/truth/mem_00{00}'.format(t) + '/free30.cice_00{00}'.format(t) + '.h.2019-04-*[0-9]*'))
        TRUTH_A = xr.open_mfdataset(truth_short, concat_dim = 'time').sel(time = slice('2019-04-15', '2019-04-30'))
        TRUTH_B = xr.open_mfdataset(truth_long, concat_dim= 'time').sel(time = slice('2019-04-15','2019-04-16'))
#     truth_files = truth_long + truth_short
#     TRUTH = xr.open_mfdataset(sorted(truth_files), concat_dim = 'time').sel(time = slice('2019-04-15', '2019-04-30'))
        experiment['TRUTH_{00}'.format(t)] = xr.concat([TRUTH_B, TRUTH_A], dim = 'time')
# truth_files_a = glob.glob(user_path + 'archive/'+free+'/ice/hist_perfect/'+ free +'_short' +'.cice_0024.h.2019-*[0-9]*')

# TRUTH = xr.open_mfdataset(truth_files, concat_dim = 'time').sel(time=slice('2019-11-01','2019-12-30'))
# TRUTH_O = xr.open_mfdataset(truth_files_a, concat_dim = 'time')
# TRUTH_A = xr.open_mfdataset(sorted(truth_files), concat_dim = 'time').sel(time = slice('2019-04-15', '2019-04-30'))
# TRUTH_L = xr.open_mfdataset(truth_files, concat_dim = 'time').sel(time = slice('2019-10-31', '2019-12-31'))

# EXPERIMENT1["TRUTH"] = xr.concat([TRUTH_A,TRUTH_O,TRUTH_L], dim="time")

for variable in variables:
    count = 0
    for x in cases:
#         print(x)
        if x == 'free30':
            data_file = glob.glob(user_path + 'archive/' + x + time_path + x +'.cice.h.' + variable + '.20150101-20191230.nc')
        else:
            data_file = glob.glob(user_path + 'archive/' + x + time_path + x +'.cice.h.' + variable + '.20190414-*[0-9]*.nc')
    
        # Trim
        DATA = xr.open_dataset(data_file[0]).sel(time = slice('2019-04-15', '2019-04-30'))
    
        # Load 
        experiment[labels[count]] = DATA

        # Advance 
        count += 1
        

    with open('experiment_{0}.p'.format(variable), 'wb') as fp:
        pickle.dump(experiment, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
    
# T = xr.Dataset.from_dict(experiment)
# T.to_netcdf('experiments')